# Worksheet 08

Name:  Junyi Li
UID: U13632870

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [13]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.761123337832542, 5.606097303286519, 4.672973244052401, 4.263337301943059, 4.885304624670285, 5.312813950733295, 5.897782154140424, 7.268062421811974, 3.7263547736183344, 4.2176964628120865]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [14]:
s2 = np.random.normal( 8,1,10 ).tolist()
print(s2)

[7.947198560989889, 8.440322359443078, 7.103084233255676, 7.251937582771932, 7.720156938405643, 8.000448429826323, 7.1722594575913, 8.678276505170833, 7.1218556217597095, 9.861594890520134]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [15]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[4.2176964628120865, 3.7263547736183344, 9.861594890520134, 7.1218556217597095, 8.678276505170833, 7.268062421811974, 5.897782154140424, 5.312813950733295, 4.885304624670285, 4.263337301943059]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Means，Variance, Mixture weight

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [18]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[9.861594890520134, 7.1218556217597095, 8.678276505170833, 7.268062421811974]
[4.2176964628120865, 3.7263547736183344, 5.897782154140424, 5.312813950733295, 4.885304624670285, 4.263337301943059]
P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 8.232447359815664,  mean_2 = 4.7172148779862475
var_1 = 1.2540844052337798,  var_2 = 0.5356764599140794


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [19]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  4.2176964628120865
probability of observing that point if it came from cluster 0 =  0.0018929212374552645
probability of observing that point if it came from cluster 1 =  0.48215389052551993
point =  3.7263547736183344
probability of observing that point if it came from cluster 0 =  0.0005001271639897412
probability of observing that point if it came from cluster 1 =  0.13459628030438725
point =  9.861594890520134
probability of observing that point if it came from cluster 0 =  0.13681303900659822
probability of observing that point if it came from cluster 1 =  6.999002803574721e-21
point =  7.1218556217597095
probability of observing that point if it came from cluster 0 =  0.21492416855842994
probability of observing that point if it came from cluster 1 =  3.135355505013488e-05
point =  8.678276505170833
probability of observing that point if it came from cluster 0 =  0.29863445460275345
probability of observing that point if it came from cluster 1 =  9.96968760663154e-13
poi

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [20]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 8.00405548782148,  mean_2 = 4.637548940055826
var_1 = 1.6508364694569102,  var_2 = 0.4757337530480862


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [21]:

updated_prob_s0_x = []  
updated_prob_s1_x = [] 
for p in data:
    pdf_i = []
    for j in range(k):
        pdf_i.append(norm.pdf(p, mean[j], (var[j]**0.5)))
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]
    updated_prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    updated_prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

updated_prob_s0_x, updated_prob_s1_x


([0.005571609464995304,
  0.003344169354244914,
  0.9999999999972242,
  0.9946394491132621,
  0.999999886909515,
  0.9977191822561043,
  0.33136116339894034,
  0.06054101797394375,
  0.01966596447148026,
  0.005949247194320306],
 [0.9944283905350046,
  0.9966558306457551,
  2.7757990323331187e-12,
  0.005360550886737982,
  1.1309048499134344e-07,
  0.0022808177438957333,
  0.6686388366010596,
  0.9394589820260563,
  0.9803340355285197,
  0.9940507528056797])

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [22]:

hard_assignments = [0 if prob_s1 >= prob_s2 else 1 for prob_s1, prob_s2 in zip(updated_prob_s0_x, updated_prob_s1_x)]
hard_assignments


[1, 1, 0, 0, 0, 0, 1, 1, 1, 1]